In [12]:
from ctypes import c_int16, c_int32

def barrett_reduce(a:int) -> int:
    assert(type(a) == int)
    a1 = c_int16()
    t = c_int16()
    m1 = c_int32()
    m2 = c_int16()
    v = c_int16()
    v1 = c_int32()
    a1.value = a
    v.value = int(((1<<26) + 1664)/3329)
    v1.value = v.value * a1.value
    m1.value = v1.value + (1<<25)
    m2.value = m1.value >> 26
    t.value = m2.value * 3329

    return a1.value - t.value

barrett_reduce(12345)

-971

In [13]:
from ctypes import c_int16, c_int32
def montgomery_reduce(a:int) -> int:
    assert(type(a) == int)
    a1 = c_int32()
    m = c_int16()
    m1 = c_int32()
    t = c_int16()
    a1.value = a
    m.value = a1.value * -3327
    m1.value = m.value * 3329
    t.value = ((a1.value - m1.value)) >> 16
    return t.value

t = montgomery_reduce(-123)
print(type(t))
# t
print(t)

<class 'int'>
-813


In [14]:
from ctypes import c_int16, c_int32
from zetas import zetas

def fqmul(a:int, b:int) -> int:
    return montgomery_reduce(a*b)

# this FINALLY works
def ntt(r:list) -> list:
    a = r
    len = 256
    k = 1
    while(len >= 4):
        len = len >> 1
        for start in range(0, 256, 2*len):
            zeta = zetas[k]
            k = k + 1
            for j in range(start, start+len, 1):
                t = fqmul(zeta, a[j + len])
                a[j + len] = a[j] - t
                a[j] = a[j] + t
    return a

t = []
for i in range(256):
    t.append(i)
n = ntt(t)

test = [5758, -484, 3754, -2534, 5194, -1973, 7282, 31, 
2483, 2197, 2725, -661, 2707, 3846, 4817, 2194, 
1971, 4132, 4251, 3560, 2319, 613, 1075, 3935, 
3635, -186, 4709, 2718, 4484, 531, 7476, -1743,
2874, 155, 304, -1887, -710, -1617, -1160, -4499,
1479, -695, -465, -1315, 1679, -129, 3431, 1923, 
1603, 558, 681, 316, 517, 931, -1597, 1999,
2024, 1094, 2276, -1170, 2187, -1356, 2637, -4500,
5702, 198, 2986, -3082, 1482, -2880, 4486, -5368, 
1057, -1109, 4453, 1019, 3729, 2206, 1225, -1096, 
1376, -449, 2664, -2715, -1369, 1974, -395, -650,
-469, 2217, -3761, -95, 1905, 3365, -1023, 2145,
219, 581, -329, -1951, -4266, 2835, -1644, 1091, 
-2275, -1179, 543, -137, -4140, 3246, -1052, 570,
239, -807, -2491, -1339, 126, -692, 126, 818, 
3232, -2254, 940, 742, 2617, -2699, 3979, -553,
-4052, 482, -4450, 868, -4709, 2149, -6921, 5225, 
-3662, -1023, -3266, 883, -866, 1313, -1378, 2999,
-3232, -1523, -3828, -1225, -1558, -876, 370, -724, 
871, 1467, -903, 1985, -966, 658, -2314, -2674, 
-2828, -2665, -2080, -209, 106, -229, 1274, 1919,
1890, -1182, -1368, -1380, 1738, 461, 2772, -2059,
-234, -1240, -2278, -4082, -1701, -1594, -3463, -4624, 
-2674, -2805, -134, -5757, 2007, -1910, 157, -4324,
2344, -504, 634, 850, -806, 2642, 672, 1604,
216, 3280, -2012, 4234, 1165, 1532, 3059, 4106, 
-3087, -1577, -1277, 533, -2323, -1471, -993, 1183,
-1673, -1661, -4621, -383, -1145, 1048, -3225, -504, 
-2452, 111, -1966, 1989, -1334, -2670, 12, 506,
-1778, -1307, -3446, 1591, -1692, 2330, -1704, -534,
-2555, 70, -2327, 3194, -2401, 987, -3941, 3005, 
-7104, -3180, -4064, -224, -827, 2134, -3941, -1026]
for i in range(len(test)):
    if(t[i]!=test[i]):
        print("[+]")
# for i in n:
#     print(i)
# n[0]
print("[+] done")

[+] done


In [15]:
from zetas import zetas
# okay this also works
def invntt(r:list) -> list:
    a = r
    len = 1
    k = 127
    while(len <= 64):
        len = len << 1
        for start in range(0, 256, 2*len):
            zeta = zetas[k]
            k = k - 1
            for j in range(start, start+len, 1):
                t = a[j]
                a[j] = barrett_reduce(t + a[j + len])
                a[j + len] = a[j + len] - t
                a[j + len] = fqmul(zeta, a[j + len])
    for i in range(256):
        a[i] = fqmul(a[i], 1441)
    return a

t = []
for i in range(256):
    t.append(i)
n = invntt(t)

test = [572, -472, 249, 249, -837, -837, 1401, 1401, 
-1442, -1442, -855, -855, -513, -513, -809, -809,
-1365, -1365, -687, -687, 979, 979, 1031, 1031, 
-71, -71, 463, 463, 365, 365, -298, -298, 
300, 300, -1207, -1207, -820, -820, -1655, -1655, 
583, 583, -1315, -1315, 1006, 1006, -466, -466, 
1214, 1214, 1385, 1385, -427, -427, -891, -891, 
-110, -110, -174, -174, -216, -216, -1347, -1347,
-55, -55, -733, -733, 671, 671, -792, -792,
-1340, -1340, 61, 61, 339, 339, -1303, -1303, 
339, 339, 1228, 1228, 606, 606, 1239, 1239,
-185, -185, 495, 495, -602, -602, -834, -834,
56, 56, -1511, -1511, -637, -637, 1633, 1633, 
1034, 1034, 919, 919, -484, -484, 138, 138,
-1380, -1380, -131, -131, 1209, 1209, -1101, -1101,
787, 787, -979, -979, -95, -95, -269, -269,
1423, 1423, -269, -269, -95, -95, -979, -979, 
787, 787, -1101, -1101, 1209, 1209, -131, -131,
-1380, -1380, 138, 138, -484, -484, 919, 919,
1034, 1034, 1633, 1633, -637, -637, -1511, -1511, 
56, 56, -834, -834, -602, -602, 495, 495,
-185, -185, 1239, 1239, 606, 606, 1228, 1228,
339, 339, -1303, -1303, 339, 339, 61, 61,
-1340, -1340, -792, -792, 671, 671, -733, -733, 
-55, -55, -1347, -1347, -216, -216, -174, -174,
-110, -110, -891, -891, -427, -427, 1385, 1385,
1214, 1214, -466, -466, 1006, 1006, -1315, -1315, 
583, 583, -1655, -1655, -820, -820, -1207, -1207,
300, 300, -298, -298, 365, 365, 463, 463,
-71, -71, 1031, 1031, 979, 979, -687, -687, 
-1365, -1365, -809, -809, -513, -513, -855, -855,
-1442, -1442, 1401, 1401, -837, -837, 249, 249]
for i in range(len(t)):
    if(t[i]!=test[i]):
        print("[+]")
print(n)
print("[+] done")

[572, -472, 249, 249, -837, -837, 1401, 1401, -1442, -1442, -855, -855, -513, -513, -809, -809, -1365, -1365, -687, -687, 979, 979, 1031, 1031, -71, -71, 463, 463, 365, 365, -298, -298, 300, 300, -1207, -1207, -820, -820, -1655, -1655, 583, 583, -1315, -1315, 1006, 1006, -466, -466, 1214, 1214, 1385, 1385, -427, -427, -891, -891, -110, -110, -174, -174, -216, -216, -1347, -1347, -55, -55, -733, -733, 671, 671, -792, -792, -1340, -1340, 61, 61, 339, 339, -1303, -1303, 339, 339, 1228, 1228, 606, 606, 1239, 1239, -185, -185, 495, 495, -602, -602, -834, -834, 56, 56, -1511, -1511, -637, -637, 1633, 1633, 1034, 1034, 919, 919, -484, -484, 138, 138, -1380, -1380, -131, -131, 1209, 1209, -1101, -1101, 787, 787, -979, -979, -95, -95, -269, -269, 1423, 1423, -269, -269, -95, -95, -979, -979, 787, 787, -1101, -1101, 1209, 1209, -131, -131, -1380, -1380, 138, 138, -484, -484, 919, 919, 1034, 1034, 1633, 1633, -637, -637, -1511, -1511, 56, 56, -834, -834, -602, -602, 495, 495, -185, -185, 1239, 12